In [443]:
import sys
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.reset_option('^display.', silent=True)

In [444]:
ratings = pd.read_csv("ratings.csv")
remarks = pd.read_csv("remarks.csv")
remarks_supp_opp = pd.read_csv("remarks_supp_opp.csv")
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

C:\Users\Naman\anaconda3\envs\prml\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [445]:
# Define all the functions here
# Use these for creating features in both train and test set

def add_latest_rating(X, ratings):
    l = []
    ratings['Date'] = pd.to_datetime(ratings.Date)
    for i in range(X.shape[0]):
        comp = X.loc[i,"comp"]
        emp = X.loc[i,"emp"]
        date = X.loc[i,"lastratingdate"]
        latest_rating = ratings[(ratings['comp']==comp) & (ratings['emp']==emp) & (ratings['Date']==date)]
        if latest_rating.empty==False:
            l.append(latest_rating.iloc[0]['rating'])
        else: 
            l.append(-1)
    X['rating'] = l
    return

def add_avg_rating_and_total_ratings(X, ratings):
    total = []
    avg = []
    for i in range(X.shape[0]):
        comp = X.loc[i,"comp"]
        emp = X.loc[i,"emp"]
        latest_ratings = ratings[(ratings['comp']==comp) & (ratings['emp']==emp)]
        if latest_ratings.empty==False:
            total.append(latest_ratings.shape[0])
            temp = latest_ratings['rating'].sum()
            temp = temp/latest_ratings.shape[0]
            avg.append(temp)
        else: 
            total.append(0)
            avg.append(0)
    X['no_of_ratings'] = total
    X['avg_rating'] = avg
    return

def add_avg_remark_length_and_total_remarks(X, remarks):
    remarks['remarks_length'] = remarks['txt'].str.len()
    no_of_remarks = []
    avg_remark_length = []
    for i in range(X.shape[0]):
        comp = X.loc[i,"comp"]
        emp = X.loc[i,"emp"]
        uni_remarks = remarks[(remarks['comp']==comp) & (remarks['emp']==emp) ]
        if uni_remarks.empty==False:
            no_of_remarks.append(uni_remarks.shape[0])
            sum = uni_remarks['remarks_length'].sum()
            avg = sum/uni_remarks.shape[0]
            avg_remark_length.append(avg)
        else: 
            no_of_remarks.append(0)
            avg_remark_length.append(0)
      # print(i)
    X['no_of_remarks'] = no_of_remarks
    X['avg_remark_length'] = avg_remark_length
    return

def add_days(X):
    X['lastratingdate'] = pd.to_datetime(X.lastratingdate)
    max = X.loc[0,"lastratingdate"]
    for i in range(1,X.shape[0]):
        date = X.loc[i,"lastratingdate"]
        if date > max:
            max = date
    l = []
    for i in range(0,X.shape[0]):
        l.append(max)
    X['last'] = l
    X['diff_days'] = (X['last'] - X['lastratingdate'])/np.timedelta64(1,'D') 
    X['month'] = X['lastratingdate'].dt.month
    X['year'] = X['lastratingdate'].dt.year

In [446]:
company_list={}
company_id = {}
id = 1
for i,comp in enumerate(train['comp']):
    if comp in company_list:
        company_list[comp][train.iloc[i]['left']]+=1
    else:
        company_id[comp] = id
        id += 1
        z=[0,0]
        z[train.iloc[i]['left']]+=1
        company_list[comp]=z
comp_rep={}
for comp in company_list:
    comp_rep[comp]=company_list[comp][1]/(company_list[comp][0]+company_list[comp][1])

In [447]:
comp_supp_opp={}
for i,comp in enumerate(remarks_supp_opp['comp']):
    if i%5000==0:
        print(i)
    if comp in comp_supp_opp:
        if(remarks_supp_opp.iloc[i]['support']==True):
            comp_supp_opp[comp][1] += 1
        else:
            comp_supp_opp[comp][2] += 1
        comp_supp_opp[comp][0] += 1
    else:
        z = [0,0,0]
        if(remarks_supp_opp.iloc[i]['support']==True):
            z[1] += 1
        else:
            z[2] += 1
        z[0] += 1
        comp_supp_opp[comp] = z

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000


In [561]:
comp_rep

{'phcvroct': 0.4883227176220807,
 'jblrepyr': 0.0,
 'ewpvmfbc': 0.06299212598425197,
 'wsmblohy': 0.18055555555555555,
 'ujplihug': 0.19337016574585636,
 'bnivzbfi': 0.0661764705882353,
 'rcwkfavv': 0.10465116279069768,
 'jnvpfmup': 0.0,
 'nmxkgvmi': 0.0,
 'bhqczwkj': 0.03211009174311927,
 'oecfwdaq': 0.3076923076923077,
 'spfcrgea': 0.07407407407407407,
 'azalutpt': 0.0,
 'yodaczsb': 0.7808988764044944,
 'siexkzzo': 0.02631578947368421,
 'ydqdpmvi': 0.07453416149068323,
 'ugldwwzf': 0.14912280701754385,
 'ocsicwng': 0.02702702702702703,
 'iqdwmigj': 0.03669724770642202,
 'ejeyobsm': 0.0,
 'ylpksopb': 0.18867924528301888,
 'fqsozvpv': 0.0,
 'vcqsbirc': 0.18181818181818182,
 'vwcdylha': 0.07017543859649122,
 'rujnkvse': 0.0,
 'lydqevjo': 0.0,
 'xccmgbjz': 0.23255813953488372,
 'lgqwnfsg': 0.11363636363636363,
 'ojidyfnn': 0.017543859649122806,
 'rcyiinms': 0.0,
 'dmgwoqhz': 0.02702702702702703,
 'pkeebtfe': 0.0,
 'pfmjacpm': 0.0,
 'bucyzegb': 0.0,
 'oqvaqcak': 0.0,
 'fjslutlg': 0.0,
 'z

In [449]:
add_latest_rating(train, ratings)
print("ratings added")
add_avg_rating_and_total_ratings(train, ratings)
print("avg ratings and total ratings added")
add_avg_remark_length_and_total_remarks(train, remarks)
print("avg remark length and total remarks added")
add_days(train)
print("days added")
#company rep is basically the reputation of the company and has been found out using the percentage of people in 
#that company who left that company, This 
l=[]
id=[]
for i,comp in enumerate(train['comp']):
    l.append(comp_rep[comp])
    id.append(company_id[comp])
train['company_id'] = id
train['company_reputation']=l
print("company reputation added")


ratings added
avg ratings and total ratings added
avg remark length and total remarks added
days added
company reputation added


In [450]:
remarks = remarks.sort_values(by='remarkId')
remarks_supp_opp = remarks_supp_opp.sort_values(by='remarkId')
j = 0
support = []
oppose = []
for i in range(remarks.shape[0]):
    if i%2==1:
        continue
    if i%5000 ==0:
        print(i)
    remark_id = remarks.iloc[i]["remarkId"]
    supp = 0
    opp = 0
    while (j<remarks_supp_opp.shape[0]) and (str(remarks_supp_opp.iloc[j]["remarkId"]) <= remark_id):
        if remarks_supp_opp.iloc[j]["remarkId"] == remark_id:
            if remarks_supp_opp.iloc[j]["support"] == True:
                supp += 1
            else:
                opp += 1
        j += 1
    support.append(supp)
    oppose.append(opp)
    support.append(supp)
    oppose.append(opp)
remarks['support'] = support
remarks['oppose'] = oppose
# print(remark)
## add in X(train or test)


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000


In [451]:
s = []
o = []
for i in range(train.shape[0]):
    comp = train.loc[i,"comp"]
    emp = train.loc[i,"emp"]
    match = remarks[(remarks['comp']==comp) & (remarks['emp']==emp) ]
    if match.empty==False:
        supp = match[match["support"]==True].shape[0]
        opp = match.shape[0] - supp
        s.append(supp)
        o.append(opp)
    else: 
        s.append(0)
        o.append(0)
train['support'] = s
train['oppose'] = o

In [452]:
train

,id,emp,comp,lastratingdate,left,rating,no_of_ratings,avg_rating,no_of_remarks,avg_remark_length,last,diff_days,month,year,company_id,company_reputation,support,oppose
0,2228,939,phcvroct,2016-10-17,1,3,1,3.000000,0,0.000000,2017-12-03,412.0,10,2016,1,0.488323,0,0
1,4349,250,jblrepyr,2017-03-19,0,4,112,3.767857,104,136.326923,2017-12-03,259.0,3,2017,2,0.000000,6,98
2,945,134,ewpvmfbc,2016-09-21,0,3,26,2.923077,10,45.800000,2017-12-03,438.0,9,2016,3,0.062992,0,10
3,4553,164,wsmblohy,2017-03-17,0,2,17,2.411765,10,21.200000,2017-12-03,261.0,3,2017,4,0.180556,1,9
4,941,129,ewpvmfbc,2016-04-04,0,3,5,2.800000,4,75.000000,2017-12-03,608.0,4,2016,3,0.062992,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3521,783,87,ocsicwng,2017-03-20,0,3,71,2.901408,18,74.444444,2017-12-03,258.0,3,2017,18,0.027027,4,14
3522,3655,14,jnvpfmup,2017-03-17,0,3,12,3.083333,4,28.500000,2017-12-03,261.0,3,2017,8,0.000000,0,4
3523,1460,53,xccmgbjz,2017-09-03,0,3,2,3.000000,0,0.000000,2017-12-03,91.0,9,2017,27,0.232558,0,0
3524,842,168,ocsicwng,2017-01-03,0,3,12,3.166667,6,82.333333,2017-12-03,334.0,1,2017,18,0.027027,2,4


In [453]:
add_latest_rating(test, ratings)
print("ratings added")
add_avg_rating_and_total_ratings(test, ratings)
print("avg ratings and total ratings added")
add_avg_remark_length_and_total_remarks(test, remarks)
print("avg remark length and total remarks added")
add_days(test)
print("days added")


l=[]
id=[]
for i,comp in enumerate(test['comp']):
    l.append(comp_rep[comp])
    id.append(company_id[comp])
test['company_id'] = id
test['company_reputation']=l

s = []
o = []
for i in range(test.shape[0]):
    comp = test.loc[i,"comp"]
    emp = test.loc[i,"emp"]
    match = remarks[(remarks['comp']==comp) & (remarks['emp']==emp) ]
    if match.empty==False:
        supp = match[match["support"]==True].shape[0]
        opp = match.shape[0] - supp
        s.append(supp)
        o.append(opp)
    else: 
        s.append(0)
        o.append(0)
test['support'] = s
test['oppose'] = o

ratings added
avg ratings and total ratings added
avg remark length and total remarks added
days added


In [454]:
test

,id,emp,comp,lastratingdate,rating,no_of_ratings,avg_rating,no_of_remarks,avg_remark_length,last,diff_days,month,year,company_id,company_reputation,support,oppose
0,353,509,bnivzbfi,2017-03-20,4,43,4.000000,6,86.000000,2017-12-01,256.0,3,2017,6,0.066176,0,6
1,732,22,ocsicwng,2017-03-17,2,69,2.130435,10,43.800000,2017-12-01,259.0,3,2017,18,0.027027,4,6
2,3076,2,siexkzzo,2017-09-01,3,18,2.444444,0,0.000000,2017-12-01,91.0,9,2017,15,0.026316,0,0
3,2199,885,phcvroct,2016-01-10,1,83,2.831325,4,67.500000,2017-12-01,691.0,1,2016,1,0.488323,0,4
4,2145,756,phcvroct,2017-06-02,3,9,2.111111,2,78.000000,2017-12-01,182.0,6,2017,1,0.488323,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,4022,36,ydqdpmvi,2016-05-10,2,74,2.662162,2,26.000000,2017-12-01,570.0,5,2016,16,0.074534,0,2
878,3397,30,fqsozvpv,2017-03-14,4,2,4.000000,4,72.000000,2017-12-01,262.0,3,2017,22,0.000000,3,1
879,1028,15,rujnkvse,2017-01-30,4,4,3.250000,0,0.000000,2017-12-01,305.0,1,2017,25,0.000000,0,0
880,3179,127,siexkzzo,2017-03-17,3,27,2.814815,0,0.000000,2017-12-01,259.0,3,2017,15,0.026316,0,0


In [479]:
train_naman = pd.read_csv("train_naman.csv")
test_naman = pd.read_csv("test_naman.csv")
train_final = pd.read_csv("train_final (1).csv")
test_final = pd.read_csv("test_final (1).csv")

In [489]:
train_naman['rem_sup_type'] = train_final['rem_sup_type']
train_naman['rem_opp_type'] = train_final['rem_opp_type']
train_naman['last_2_months'] = train_final['last_2_months']
train_naman['last_6_months'] = train_final['last_6_months']
train_naman['last_12_months'] = train_final['last_12_months']
train_naman['num_ratings_2'] = train_final['num_ratings_2']
train_naman['num_ratings_6'] = train_final['num_ratings_6']

test_naman['rem_sup_type'] = test_final['rem_sup_type']
test_naman['rem_opp_type'] = test_final['rem_opp_type']
test_naman['last_2_months'] = test_final['last_2_months']
test_naman['last_6_months'] = test_final['last_6_months']
test_naman['last_12_months'] = test_final['last_12_months']
test_naman['num_ratings_2'] = test_final['num_ratings_2']
test_naman['num_ratings_6'] = test_final['num_ratings_6']

In [546]:
test_naman.head(3)

,id,emp,comp,lastratingdate,rating,no_of_ratings,avg_rating,no_of_remarks,avg_remark_length,last,diff_days,month,year,company_id,company_reputation,support,oppose,rem_sup_type,rem_opp_type,last_2_months,last_6_months,last_12_months,num_ratings_2,num_ratings_6
0,353,509,bnivzbfi,2017-03-20,4,43,4.000000,6,86.0,2017-12-01,256.0,3,2017,6,0.066176,0,6,103.129870,2.636364,4.0,4.000000,4.000000,2,10
1,732,22,ocsicwng,2017-03-17,2,69,2.130435,10,43.8,2017-12-01,259.0,3,2017,18,0.027027,4,6,243.340909,20.670455,2.0,1.666667,1.972973,2,6
2,3076,2,siexkzzo,2017-09-01,3,18,2.444444,0,0.0,2017-12-01,91.0,9,2017,15,0.026316,0,0,111.555556,94.740741,3.0,3.000000,2.666667,1,1


In [624]:
msk = np.random.rand(len(train_naman)) < 0.75
x_train = train_naman[msk]
x_cv = train_naman[~msk]

features = ['company_id','month', 'year','last_2_months','last_6_months','last_12_months','num_ratings_2', 'num_ratings_6','rem_sup_type','rem_opp_type','diff_days','no_of_remarks','avg_remark_length','avg_rating','support','oppose']
cat = ['company_id','month']

X_train = x_train[features].to_numpy()
Y_train = x_train['left'].to_numpy()
X_cv = x_cv[features].to_numpy()
Y_cv = x_cv['left'].to_numpy()
X_test = test_naman[features].to_numpy()



In [625]:
def cal_score(y_pred,y):
    res = y + y_pred
    accuracy=(np.sum(res==2)*5+np.sum(res==0))/(np.sum(y==1)*5+np.sum(y==0))
    print(accuracy)

def get_score(clf):
    y_pred_cv=clf.predict(X_cv)
    y_pred_cv[(y_pred_cv>=0.5)]=1
    y_pred_cv[(y_pred_cv<0.5)]=0

    y_pred_tr=clf.predict(X_train)
    y_pred_tr[(y_pred_tr>=0.5)]=1
    y_pred_tr[(y_pred_tr<0.5)]=0

    cal_score(y_pred_tr,Y_train)
    cal_score(y_pred_cv,Y_cv)

In [626]:
# from sklearn.ensemble import RandomForestClassifier
# sizes = [1,2,3,4,5,6,7,8,9,10,4,8,16,25,32,50,64,128]
# # sizes = [8]
# for size in sizes:
#     print(size)
#     tree = RandomForestClassifier(n_estimators=50, criterion='gini', class_weight={1:5, 0:1}, max_depth=size+1)
#     clf = tree.fit(X_train,Y_train)
# #     y_pred_test = clf.predict(X_test)
#     get_score(clf)


In [627]:
import lightgbm as lgb
# leaves = [2,3,4,5,6,7,8,9,10,11,12,13]
leaves = [4]
for leaf_number in leaves:
    LGB = lgb.LGBMClassifier(boosting_type='gbdt', 
#                                 num_leaves=leaf_number, 
                                max_depth=leaf_number, 
                                learning_rate=0.1, 
                                n_estimators=100, 
                                objective='binary', 
                                class_weight={1:5, 0:1})
    clfLGB = LGB.fit(X_train, Y_train, feature_name=features, categorical_feature = cat)
    print(leaf_number)
    get_score(clfLGB)
#     print('\n')
y_pred_test=clfLGB.predict(X_test)
y_pred_test[(y_pred_test>=0.5)]=1
y_pred_test[(y_pred_test<0.5)]=0

4
0.9647406733393995
0.897078353253652


C:\Users\Naman\anaconda3\envs\prml\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['company_id', 'month']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


In [472]:
# from catboost import CatBoostClassifier
# model = CatBoostClassifier(class_weights={1:5, 0:1},
#                            max_depth=8, 
#                            n_estimators=100,
#                            learning_rate=0.01, 
#                            min_data_in_leaf = 50,
#         a                   loss_function='Logloss')
# model.fit(X_train, Y_train,cat_features=[0],silent=True)
# y_pred_test=model.predict(X_test)
# y_pred_test[(y_pred_test>=0.5)]=1
# y_pred_test[(y_pred_test<0.5)]=0
# get_score(model)

In [471]:
test['left'] = y_pred_test.tolist()
result = test[['id','left']]
result.to_csv('result.csv',index=False)

In [491]:
# train_naman.to_csv('train_naman.csv', index=False)
# test_naman.to_csv('test_naman.csv', index=False)

In [628]:
y_pred_cv = clfLGB.predict(X_cv)
y_pred_cv[(y_pred_cv>=0.5)]=1
y_pred_cv[(y_pred_cv<0.5)]=0
res = np.zeros(37)
tot = np.zeros(37)
for i in range(X_cv.shape[0]):
    tot[int(X_cv[i][0])-1] += 1
    if not y_pred_cv[i]==Y_cv[i]:
        res[int(X_cv[i][0])-1] += 1
print(tot)
print(res)

[119.  42.  37.  56.  49.  68.  60.   6.   5.  61.   1.   3.   1.  54.
  61.  40.  23.  29.  31.  23.  15.   9.   3.  29.  21.   3.  10.  13.
  13.  10.  12.   6.   3.   2.   6.   2.   0.]
[28.  0.  4.  2. 15.  6.  4.  0.  0.  3.  1.  1.  0.  8.  2.  0.  1.  0.
  1.  0.  5.  0.  1.  1.  0.  0.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.]


In [592]:
27/111

0.24324324324324326

In [637]:
comp_1 = train_naman[train_naman['company_id']==1]

In [638]:
comp_1

,id,emp,comp,lastratingdate,left,rating,no_of_ratings,avg_rating,no_of_remarks,avg_remark_length,last,diff_days,month,year,company_id,company_reputation,support,oppose,rem_sup_type,rem_opp_type,last_2_months,last_6_months,last_12_months,num_ratings_2,num_ratings_6
0,2228,939,phcvroct,2016-10-17,1,3,1,3.000000,0,0.0,2017-12-03,412.0,10,2016,1,0.488323,0,0,-100.000000,-100.000000,3.000000,3.000000,3.000000,1,1
18,2135,726,phcvroct,2016-10-08,1,3,12,3.666667,0,0.0,2017-12-03,421.0,10,2016,1,0.488323,0,0,273.000000,0.000000,3.666667,3.571429,3.666667,3,7
31,1973,349,phcvroct,2017-01-16,0,4,1,4.000000,0,0.0,2017-12-03,321.0,1,2017,1,0.488323,0,0,-100.000000,-100.000000,4.000000,4.000000,4.000000,1,1
37,2115,655,phcvroct,2016-06-28,0,1,4,1.500000,2,39.0,2017-12-03,523.0,6,2016,1,0.488323,0,2,-100.000000,-100.000000,3.000000,1.500000,1.500000,1,4
44,2029,470,phcvroct,2015-08-21,1,3,1,3.000000,0,0.0,2017-12-03,835.0,8,2015,1,0.488323,0,0,123.000000,0.000000,3.000000,3.000000,3.000000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3465,2085,595,phcvroct,2016-07-25,1,4,102,3.960784,10,66.8,2017-12-03,496.0,7,2016,1,0.488323,1,9,93.451613,12.451613,4.000000,4.000000,3.960784,7,37
3470,2064,558,phcvroct,2017-01-24,0,3,150,2.926667,0,0.0,2017-12-03,313.0,1,2017,1,0.488323,0,0,-100.000000,-100.000000,3.000000,3.000000,2.925926,2,4
3471,1975,353,phcvroct,2015-03-30,0,3,1,3.000000,0,0.0,2017-12-03,979.0,3,2015,1,0.488323,0,0,-100.000000,-100.000000,3.000000,3.000000,3.000000,1,1
3488,2159,813,phcvroct,2016-12-05,0,3,1,3.000000,2,30.0,2017-12-03,363.0,12,2016,1,0.488323,1,1,32.000000,0.000000,3.000000,3.000000,3.000000,1,1


In [629]:
def cal_score(y_pred,y):
    res = y + y_pred
    accuracy=(np.sum(res==2)*5+np.sum(res==0))/(np.sum(y==1)*5+np.sum(y==0))
    print(accuracy)

def get_score(clf):
    y_pred_cv=clf.predict(x_cv_comp_1)
    y_pred_cv[(y_pred_cv>=0.5)]=1
    y_pred_cv[(y_pred_cv<0.5)]=0

    y_pred_tr=clf.predict(x_tr_comp_1)
    y_pred_tr[(y_pred_tr>=0.5)]=1
    y_pred_tr[(y_pred_tr<0.5)]=0

    cal_score(y_pred_tr,y_tr_comp_1)
    cal_score(y_pred_cv,y_cv_comp_1)

In [630]:
x_tr_comp_1 = X_train[X_train[:,0]==1] 
x_cv_comp_1 = X_cv[X_cv[:,0]==1]
y_tr_comp_1 = Y_train[X_train[:,0]==1]
y_cv_comp_1 = Y_cv[X_cv[:,0]==1]

In [635]:
leaves = [2,3,4,5,6,7,8,9,10,11,12,13]
# leaves = [2]
for leaf_number in leaves:
    LGB_comp_1 = lgb.LGBMClassifier(boosting_type='gbdt', 
                                num_leaves=leaf_number, 
#                                 max_depth=leaf_number, 
                                learning_rate=0.1, 
                                n_estimators=100, 
                                objective='binary', 
                                class_weight={1:5, 0:1})
    clfLGB_comp_1 = LGB_comp_1.fit(x_tr_comp_1, y_tr_comp_1, feature_name=features, categorical_feature = cat)
    print(leaf_number)
    get_score(clfLGB_comp_1)
#     print('\n')

C:\Users\Naman\anaconda3\envs\prml\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['company_id', 'month']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\Naman\anaconda3\envs\prml\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['company_id', 'month']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\Naman\anaconda3\envs\prml\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['company_id', 'month']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\Naman\anaconda3\envs\prml\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['company_id', 'month']
  'New categorical_fe

2
0.8953488372093024
0.8746518105849582
3
0.9263565891472868
0.8774373259052924
4
0.9408914728682171
0.8495821727019499
5
0.9428294573643411
0.8662952646239555
6
0.9534883720930233
0.8551532033426184
7
0.9602713178294574
0.8272980501392758
8
0.9689922480620154
0.8495821727019499
9
0.9738372093023255
0.8356545961002786
10
0.9757751937984496
0.8189415041782729
11
0.9777131782945736
0.7910863509749304
12
0.9796511627906976
0.8022284122562674


C:\Users\Naman\anaconda3\envs\prml\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['company_id', 'month']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\Naman\anaconda3\envs\prml\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['company_id', 'month']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\Naman\anaconda3\envs\prml\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['company_id', 'month']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\Naman\anaconda3\envs\prml\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['company_id', 'month']
  'New categorical_fe

13
0.9786821705426356
0.7799442896935933


In [636]:
# print(y_pred)
y_pred_cv = clfLGB_comp_1.predict(x_cv_comp_1)
y_pred_cv[(y_pred_cv>=0.5)]=1
y_pred_cv[(y_pred_cv<0.5)]=0
res = np.zeros(37)
tot = np.zeros(37)
for i in range(x_cv_comp_1.shape[0]):
    tot[int(x_cv_comp_1[i][0])-1] += 1
    if not y_pred_cv[i]==y_cv_comp_1[i]:
        res[int(x_cv_comp_1[i][0])-1] += 1
        print(y_pred_cv[i])
print(tot)
print(res)
print(y_cv_comp_1)
print(y_pred_cv)

1
0
0
0
0
0
1
1
1
0
0
1
0
1
1
1
1
1
1
0
1
1
1
1
0
1
0
1
1
0
1
[119.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.]
[31.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.]
[0 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1 1 0 1 1 0 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1
 1 0 1 0 0 1 0 0 0 1 1 1 1 1 1 0 1 0 1 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0
 0 1 0 1 1 0 0 1 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 0 1 0 1 0 1 0 1 0 0 1 0 0 1
 1 0 0 0 1 1 1 0]
[1 1 1 0 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1 0 0 1 0 1 1
 0 0 1 0 0 1 0 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 1 0 0 1 1 1 1 0 0 0
 1 0 0 1 1 0 0 1 0 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 1 0 1 0 1 1 1 0 0 1 0 0 0
 1 1 0 1 0 1 1 1]
